# Repo

In [ ]:
#@title Clone repo


!git clone https://github.com/AndyRightNow/fish-speech.git
%cd fish-speech

In [ ]:
#@title Checkout and pull code

!git checkout andy
!git pull
#!git fetch --all && git reset --hard origin/andy

In [ ]:
#@title Install `uv`

!curl -LsSf https://astral.sh/uv/install.sh | sh
!source $HOME/.local/bin/env

In [ ]:
#@title Install dependencies

!uv pip install --system torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 m4b-util==0.4.1


!apt install libsox-dev ffmpeg

!apt install build-essential \
    cmake \
    libasound-dev \
    portaudio19-dev \
    libportaudio2 \
    libportaudiocpp0

!uv pip install --system -e .[stable]

In [ ]:
#@title Download checkpoints

from google.colab import userdata
huggingface_token = userdata.get('huggingfaceToken')

!uv pip install --system -U "huggingface_hub[cli]"
!huggingface-cli login --token $huggingface_token  --add-to-git-credential
!huggingface-cli download fishaudio/fish-speech-1.5 --local-dir checkpoints/fish-speech-1.5

In [ ]:
#@title Upload input files

%cd /content/fish-speech/generate_pipelines
!mkdir input
%cd input

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

%cd /content/fish-speech

# Run generate script

In [ ]:
#@title __Archived scripts

# !python generate_pipelines/main.py --prompt-name tifa --input-name we-after-everything --insert-segment-title true --segment-title-language en --max-sem-input-count 2500
# !python generate_pipelines/convert_mp3.py --input-name we-after-everything --prompt-name tifa --max-sem-input-count 2500
# !python generate_pipelines/convert_mp3.py --input-name hengxingshike --prompt-name weisan

# ------------------------------------ Archived ------------------------------------


In [ ]:
# @title Main scripts
# Run scripts here

from google.colab import output

output.clear()

!LOGURU_LEVEL=SUCCESS python generate_pipelines/main.py --prompt-name weisan --input-name hengxingshike --start-segment-index 546 --generate-mp3 true

Encoded::  37% 15/41 [01:00<00:36,  1.41s/it]

In [ ]:
#@title Bind audiobook


hash = 'c701d5ede41f188c4368bd47a23bfce814c56fad353467026d67a719aae98c22' # @param {type: "string"}
title = 'Hengxingshike Part 5' # @param {type: "string"}
author = '稚楚' # @param {type: "string"}

!python -m m4b_util bind generate_pipelines/output/mp3/$hash -o generate_pipelines/output -t "$title" --use-filename -a "$author"

In [ ]:
#@title Download audiobook

from google.colab import files


if title and author:
  files.download(f'generate_pipelines/output/{author} - {title}.m4b')

In [ ]:
#@title Download states

# prompt: use git to check if there are any modified files under generate_pipelines/states folder, and if so, zip generate_pipelines/states folder and download the zip folder with the name "states_{time}.zip". If no file changes, log to console

import datetime
import os

# Check for modified files in generate_pipelines/states
!git status -s generate_pipelines/states

# Check if there's any output from the git status command
if os.system("git status -s generate_pipelines/states | grep -q . || exit 1") == 1:
  print("No changes detected in generate_pipelines/states")
else:
  # Get the current time
  now = datetime.datetime.now()
  timestamp = now.strftime("%Y%m%d_%H%M%S")

  # Zip the folder
  zip_filename = f"states_{timestamp}.zip"
  !zip -r $zip_filename generate_pipelines/states

  # Download the zip file
  from google.colab import files
  files.download(zip_filename)

In [ ]:
#@title (Dangerous) Hard reset to origin

!git checkout andy

confirm = input("Are you sure?(y/n)")

if confirm == "y":
  !git fetch --all && git reset --hard origin/andy

# Debug

In [ ]:
debug = True  # @param {type: "boolean"}

In [ ]:
if debug:
  %cd /content/fish-speech
  !python generate_pipelines/debug_print_shelve.py --input-name hengxingshike --prompt-name weisan

  !cat generate_pipelines/states/states.json